In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from PIL import ImageFont, ImageDraw, Image, ImageTk


happy = ""
    
# Define actions and corresponding labels
actions = ['ㄱ','ㄴ','ㄷ','ㄹ','ㅁ','ㅂ','ㅅ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ','ㅏ','ㅑ','ㅓ','ㅕ','ㅗ','ㅛ','ㅜ','ㅠ','ㅡ','ㅣ','ㅐ','ㅒ','ㅔ','ㅖ','ㅚ','ㅟ','ㅢ','space','backspace','1','2','3']
seq_length = 30

model = load_model('models/models.h5')

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1680)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1280)

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    

    if result.multi_hand_landmarks is not None:
    
        for res in result.multi_hand_landmarks:
            
            joint = np.zeros((21, 4))
            
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                
            v1 = joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19], :3]
            v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], :3]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            angle = np.arccos(np.einsum('nt,nt->n', v[[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18], :],
                                        v[[1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19], :]))
            angle = np.degrees(angle)

            d = np.concatenate([joint.flatten(), angle])
            seq.append(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) < seq_length:
                
                print("1")
                continue

            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)
            y_pred = model.predict(input_data).squeeze()

            i_pred = int(np.argmax(y_pred))  
            conf = y_pred[i_pred]
            
            if conf < 0.8:
                print("2")
                continue

            action = actions[i_pred]
            action_seq.append(action)
            print("2.5")

            if len(action_seq) < 3:
                print("3")
                continue

            this_action = '?'
            if action_seq[-1] == action_seq[-2] == action_seq[-3]: # 정확한 액션이라면
                this_action = action
                
            
            print(this_action)
            
            if cv2.waitKey(1) == ord('a'):
                if this_action == 'space':
                    this_action = " "
                    happy = happy + this_action
                    
                if this_action =='backspace':
                    happy = happy[:-1]
                    print(happy)
                    
                else:
                    happy = happy + this_action # upper() 대문자 변환 함수
            print(type(happy))
            
            font = ImageFont.truetype('C:/Users/kelly/IdeaProjects/Project mid/font/MaruBuri-Bold.ttf',35)       
            img = Image.fromarray(img)
            draw = ImageDraw.Draw(img)
            org = (25,50)
            
            draw.text(org, f'{this_action}', font=font, fill=(0,0,0))
            draw.text((25,600), happy, font=font, fill=(0,0,0))
    
            img = np.array(img)
            cv2.imshow('happy', img)
            
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1/1 [==============================] - 0s 146ms/step
2.5
3
1/1 [==============================] - 0s 24ms/step
2.5
3
1/1 [==============================] - 0s 23ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 26ms/step
2
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 22ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 24ms/st

1/1 [==============================] - 0s 27ms/step
2.5
ㄴ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄴ
<class 'str'>
1/1 [==============================] - 0s 26ms/step
2.5
ㄴ
<class 'str'>
1/1 [==============================] - 0s 26ms/step
2.5
ㄴ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㄴ
<class 'str'>
1/1 [==============================] - 0s 27ms/step
2.5
ㄴ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
ㄴ
<class 'str'>
1/1 [==============================] - 0s 27ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 28ms/step
2
1/1 [==============================] - 0s 22ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 27ms/step
2.5
ㄴ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 25ms/step
2
1/1 [==============================] - 0s 21ms

1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 24ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 26ms/step
2
1/1 [==============================] - 0s 25ms/step
2
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 37ms/step
2
1/1 [==============================] - 0s 22ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 21ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 28ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 28ms/step
2.5
backspace
<class 'str'>
1/1 [==============================] - 0s 26ms/step
2.5
backspace
<class 'str'>
1/1 [=====

1/1 [==============================] - 0s 27ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 26ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 34ms/step
2.5
ㅡ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㅡ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㅡ
<class 'str'>
1/1 [==============================] - 0s 31ms/step
2.5
ㅡ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㅡ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 29ms/step
2.5
ㅡ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㅡ
<class 'str'>
1/1 [==============================] - 0s 28ms/step
2
1/1 [==============================] - 0s 29ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
?
<class 'str'>
1/1 [=======================

1/1 [==============================] - 0s 31ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 21ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 21ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 28ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 30ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class

...

1/1 [==============================] - 0s 22ms/step
2.5
ㅕ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㅕ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㅕ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㅕ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㅕ
<class 'str'>
1/1 [==============================] - 0s 20ms/step
2.5
ㅕ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 22ms/step
2.5
ㅕ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 24ms/step
2.5
ㅕ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 22ms/step
2.5
ㅕ
<class 'str'>
1/1 [=======================

1/1 [==============================] - 0s 23ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 21ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 21ms/step
2
1/1 [==============================] - 0s 22ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 19ms/step
2
1/1 [==============================] - 0s 21ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class 'str'>
1/1 [=======================

1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 30ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 30ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 30ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 26ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 19ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 34ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 21ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㄱ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㄱ
<class

1/1 [==============================] - 0s 32ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 32ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 35ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 20ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 32ms/step
2.5
ㄹ
<class 'str'>
1/1 [==============================] - 0s 27ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 20ms/step
2.5
?
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 24ms/step
2.5
ㄱ
<class 'str'>
1/1 [=====

1/1 [==============================] - 0s 26ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 29ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 23ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 24ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 31ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 26ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 21ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 30ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 32ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 30ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 22ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 31ms/step
2.5
ㅛ
<class 'str'>
1/1 [==============================] - 0s 25ms/step
2.5
ㅛ
<class